In [2]:
import numpy as np
import pandas as pd
import re
import cobra
import cobra.test
from cobra import Reaction, Metabolite
from cobra.flux_analysis import gapfill, production_envelope, flux_variability_analysis
from cobra.flux_analysis.gapfilling import GapFiller
from icecream import ic

In [3]:
model = cobra.test.create_test_model("ecoli")
model_e = model.copy()

In [4]:
model

Name,iJO1366
Memory address,0x0109258490
Number of metabolites,1805
Number of reactions,2583
Objective expression,-1.0*Ec_biomass_iJO1366_core_53p95M_reverse_e94eb + 1.0*Ec_biomass_iJO1366_core_53p95M
Compartments,"Periplasm, Cytoplasm, Extracellular"


for reaction in model_text.reactions[:5]:
    with model_text as model:
        reaction.knock_out()
        model.optimize()
        print('%s blocked (bounds: %s), new growth rate %f' %
              (reaction.id, str(reaction.bounds), model.objective.value))

In [11]:
sol = model.optimize()

10 loops, best of 3: 54.8 ms per loop


In [6]:
print sol.fluxes

DM_4CRSOL                         0.000219
DM_5DRIB                          0.000221
DM_AACALD                         0.000000
DM_AMOB                           0.000002
DM_MTHTHF                         0.000440
DM_OXAM                           0.000000
Ec_biomass_iJO1366_WT_53p95M      0.000000
Ec_biomass_iJO1366_core_53p95M    0.982372
EX_12ppd__R_e                     0.000000
EX_12ppd__S_e                     0.000000
EX_14glucan_e                     0.000000
EX_15dap_e                        0.000000
EX_23camp_e                       0.000000
EX_23ccmp_e                       0.000000
EX_23cgmp_e                       0.000000
EX_23cump_e                       0.000000
EX_23dappa_e                      0.000000
EX_26dap__M_e                     0.000000
EX_2ddglcn_e                      0.000000
EX_34dhpac_e                      0.000000
EX_3amp_e                         0.000000
EX_3cmp_e                         0.000000
EX_3gmp_e                         0.000000
EX_3hcinnm_

In [7]:
model.summary()

IN FLUXES             OUT FLUXES           OBJECTIVES
--------------------  -------------------  ----------------------
o2_e       17.6       h2o_e     45.6       Ec_biomass_i...  0.982
nh4_e      10.6       co2_e     19.7
glc__D_e   10         h_e        9.03
pi_e        0.948     mththf_c   0.00044
so4_e       0.248     5drib_c    0.000221
k_e         0.192     4crsol_c   0.000219
fe2_e       0.0158    amob_c     1.96e-06
mg2_e       0.00852   meoh_e     1.96e-06
ca2_e       0.00511
cl_e        0.00511
cu2_e       0.000697
mn2_e       0.000679
zn2_e       0.000335
ni2_e       0.000317
mobd_e      0.000127
cobalt2_e   2.46e-05


In [9]:
import requests
resp = requests.get('http://bigg.ucsd.edu/api/v2/models/iJO1366/genes')
res = resp.json()['results']
res[:5]
for gene in res:
    if gene['name'] == 'atoB':
        print gene
        break

{u'model_bigg_id': u'iJO1366', u'bigg_id': u'b2224', u'name': u'atoB', u'organism': u'Escherichia coli str. K-12 substr. MG1655'}


In [9]:
alc_dehydr = cobra.Reaction(id='ALCDBUT', name='Alcohol dehydrogenase (butanal)', subsystem='c')
model.add_reaction(alc_dehydr)
alc_dehydr.add_metabolites({'btcoa_c': -1, 'h_c': -1, 'nadh_c': -1, 'nad_c': 1, 'btal_c': 1})

butanol = cobra.Metabolite(id='btol_c', name='1-butanol', compartment='c', charge=0, formula='C4H9OH')
but_synth = cobra.Reaction(id='BUTSYN', name='Butanol synthesis', subsystem='c')
model.add_reaction(but_synth)
but_synth.add_metabolites({'btal_c': -1, 'h_c': -1, 'nadh_c': -1, 'nad_c': 1, butanol: 1})

In [10]:
# TODO look at reversiblilty
# TODO look at whether we need to put custom enzymes in
# STEP 0 2 accoa_c -> aacoa_c + coa_c via atoB
model.reactions.ACACT1r
# STEP 1 aacoa_c + h_c + nadh_c -> nad_c + 3hbcoa_c via Hbd1
model.reactions.HACD1
# STEP 2 3hbcoa_c => b2coa_c + h2o_c via Crt
model.reactions.ECOAH1
# TODO: Change from fad_c to nad_c 
# STEP 3 b2coa_c + fadh2_c <=> btcoa_c + fad_c via Ter
model.reactions.ACOAD1f
# STEP 4 btcoa_c + h_c + nadh_c => nad_c + btal_c via adhE (b1241)
model.reactions.ALCDBUT
# STEP 5 btal_c + h_c + nadh_c => nad_c + btol_c via adhE
model.reactions.BUTSYN

Reaction identifier,BUTSYN
Name,Butanol synthesis
Memory address,0x0115db6590
Stoichiometry,btal_c + h_c + nadh_c --> btol_c + nad_c Butanal + H+ + Nicotinamide adenine dinucleotide - reduced --> 1-butanol + Nicotinamide adenine dinucleotide
GPR,
Lower bound,0.0
Upper bound,1000.0


In [11]:
def opt(model):
    with model:
        if not ('DM_btol_c' in model.reactions):
            model.add_boundary(model.metabolites.btol_c, type='demand')
        model.objective = model.reactions.DM_btol_c
        obj = model.optimize()
        print obj.objective_value
        print model.summary()

In [12]:
opt(model)

1.79035118836
IN FLUXES        OUT FLUXES     OBJECTIVES
---------------  -------------  ---------------
nh4_e     12.5   h2o_e   47.5   DM_btol_c  1.79
glc__D_e  10     co2_e   15.2
o2_e       9.87  h_e      5.37
pi_e       5.37  btol_c   1.79
so4_e      1.79
None


In [13]:
def convert_cmpts(model, metab, from_cmpt='c', to_cmpt='e'):
    if type(metab) is cobra.Metabolite:
        metab = metab.id.rsplit('_', 1)[0]
    metab_name = metab + '_{0}'.format(to_cmpt)
    try:
        metab_cnvrt = model.metabolites.get_by_id(metab_name)
    except KeyError:
        metab_cnvrt = model.metabolites.get_by_id(metab + '_{0}'.format(from_cmpt)).copy()
        metab_cnvrt.id, metab_cnvrt.compartment = metab_name, to_cmpt
    return metab_cnvrt

In [ ]:
model.reactions.query(lambda x: med_met in x.metabolites and x.boundary)

In [14]:
def remove_exs(model):
    model_no_ex = model.copy()
    model_no_ex.remove_reactions(reactions=model_no_ex.exchanges)
    return model_no_ex
model_no_ex = remove_exs(model)

In [15]:
def add_ex(model, metab_dict, additive=False):
    rxn_dict = {}
    for med_met, conc in metab_dict.items():
        if conc < 0:
            ty = 'exchange'
            ty_name = 'EX'
        else:
            ty = 'demand'
            ty_name = 'DM'
        rxn_nm = '{0}_'.format(ty_name) + med_met.id
        if rxn_nm in model.reactions:
            rxn = model.reactions.get_by_id(rxn_nm)
            if additive:
                rxn.lower_bound = conc + rxn.lower_bound
            #    conc += rxn.upper_bound
            #    model.reactions.remove(rxn)
            #    model.add_boundary(metabolite=med_met, type='exchange', lb=conc)
            else:
                rxn.lower_bound = conc
        else:
            model.add_boundary(metabolite=med_met, type='medium-diff', reaction_id=rxn_nm, lb=conc, ub=1000)
        rxn_dict[model.reactions.get_by_id(rxn_nm)] = conc
        #print model.reactions.get_by_id(rxn_nm).upper_bound
    return rxn_dict

In [16]:
def gen_metab_dict(model, metab_nms, metab_vals=None, cnvt=True, from_cmpt='c', to_cmpt='e'):
    metabs = []
    for metab_nm in metab_nms:
        if cnvt:
            metab = convert_cmpts(model, metab_nm, from_cmpt, to_cmpt)
        else:
            metab = metab_nm
        metabs.append(metab)
    if not metab_vals:
        metab_vals = [0] * len(metabs)
    metab_conc = dict(zip(metabs, metab_vals))
    return metab_conc

In [57]:
def ecoli_to_cf(mod):
    # Delete transmembrane transport reactions
    """model = mod.copy()
    to_delete = []
    for rxn in model.reactions:
        if ('c' in rxn.compartments and 'p' in rxn.compartments) or \
            ('e' in rxn.compartments and 'p' in rxn.compartments):
            to_delete.append(rxn)
    model.remove_reactions(to_delete, remove_orphans=True)
    
    # Move periplasmic reactions to cytosol
    peri_rxns = set()
    peri_metabs = model.metabolites.query(attribute="compartment", search_function="p")
    peri_cyt_metab_map = {}
    for peri_metab in peri_metabs:
        for rxn in peri_metab.reactions:
            peri_rxns.add(rxn)
        cyt_metab = convert_cmpts(model, peri_metab, from_cmpt='p', to_cmpt='c')
        peri_cyt_metab_map[peri_metab] = cyt_metab
        
    for peri_rxn in peri_rxns:
        peri_rxn.compartments.clear()
        peri_rxn.compartments.update(set('c'))
        peri_rxn_metabs = peri_rxn.metabolites
        coeffs = peri_rxn.get_coefficients(peri_rxn_metabs)
        peri_rxn.subtract_metabolites(dict(zip(peri_rxn_metabs, coeffs)))
        peri_rxn.add_metabolites(dict(zip(map(lambda x: peri_cyt_metab_map[x], peri_rxn_metabs), coeffs)))
            
    model.remove_metabolites(metabolite_list=peri_metabs)"""
    
    # Add exchange reactions for all metabolites
    # Exchange‐reactions for non‐substrate metabolites have lb of 0 and ub of 1000 by default
    #non_sub = set(model.metabolites) - set(map(lambda x: model.reactions.get_by_id(x).metabolites.popitem()[0], model.medium.keys()))
    all_metab_ex = gen_metab_dict(model, model.metabolites, cnvt=False)
    # Exchange reactions for substrates get negative bounds to allow them to enter system
    # Normal medium substrates maintain their lower bounds (effectively unlimited for water, protons -- TODO: check assumption)
    #media = {}
    #for rxn_nm in model.medium:
    #    rxn = model.reactions.get_by_id(rxn_nm)
    #    media[rxn.metabolites.popitem()[0]] = rxn.lower_bound
    # TODO: ensure no duplicates
    #crucial = {model.metabolites.h2o_e: -1000, model.metabolites.o2_e: -1000}
    #all_metab_ex.update(crucial)

    add_ex(model, all_metab_ex)
    model.remove_reactions(['EX_glc_e', 'EX_pi_e', 'EX_mg2_e', 'EX_k_e', 'EX_mg2_e', 'EX_nh4_e'])

    # As objective function, we selected the exchange reaction which corresponds to the target metabolite 
    # for which a pathway should be determined.   
    return model

model_cf = ecoli_to_cf(model)

In [26]:
s = 0
for rxn_nm in model_cf.medium:
    rxn = model_cf.reactions.get_by_id(rxn_nm)
    if rxn.lower_bound < 0:
        s += 1
print s

20


In [20]:
# Based on MetaboTools
# flux = metConc/(cellConc*cellWeight*t*1000);
# cellConc: 10 mg/ml
# cellWeight: 500 * (10 ** -11)
# t = 24
# t: Experiment duration
def conc_to_flux(metab_conc, t=24):
    # Taken from MetaboTools, units are gDW/cell
    #cell_weight = 500 * (10 ** -11)
    # We have 10 mg/ml
    # need cells / ml
    cell_conc = 10 * (1/ 1000.0) #* (1 / cell_weight)
    flux = metab_conc / (cell_conc * t * 1000.0)
    return flux

In [58]:
def build_medium(mod):
    model = mod.copy()
    # 20 flux is 20 mol/min/gDW
    # in mM
    aa_conc = -1 * conc_to_flux(2)
    trna_conc = -1 * conc_to_flux(0.17)
    metab_conc = map(lambda x: conc_to_flux(x) * -1, [1.2, 0.85, 0.85, 0.85, 0.33, 0.27, 1.50, 1.00, 130, 10, 12, 0.33])
    print metab_conc
    # TODO: map from conc to exchange fluxes
    
    import Bio.PDB.Polypeptide
    aas = map(lambda x: x.lower(), Bio.PDB.Polypeptide.aa3)
    aa_concs = {}
    for aa in aas:
        l_aa, d_aa = None, None
        try:
            l_aa = convert_cmpts(model, model.metabolites.get_by_id(aa + '__L_c'), from_cmpt='c', to_cmpt='e')
            d_aa = convert_cmpts(model, model.metabolites.get_by_id(aa + '__D_c'), from_cmpt='c', to_cmpt='e')
        except KeyError:
            if aa == 'gly':
                l_aa = convert_cmpts(model, model.metabolites.get_by_id(aa + '_c'), from_cmpt='c', to_cmpt='e')
        if d_aa:
            aa_concs[l_aa] = aa_conc / 2.0
            aa_concs[d_aa] = aa_conc / 2.0
        else:
            aa_concs[l_aa] = aa_conc
            
    trna_concs = {convert_cmpts(model, trna_metab, from_cmpt='c', to_cmpt='e') : trna_conc for trna_metab in model.metabolites.query('trna')}
    
    metab_nms = ['atp', 'gtp', 'utp', 'ctp', 'nad', 'coa', 'spmd', 'ptrc', 'k', 'nh4', 'mg2', 'pep']
    metab_concs = gen_metab_dict(model, metab_nms, metab_conc)
        
    # Not included: RNAP & Folinic Acid & cell extract
    base_medium = {}
    base_medium.update(add_ex(model, metab_concs))
    base_medium.update(add_ex(model, aa_concs))
    base_medium.update(add_ex(model, trna_concs))
    
    #model.medium = base_medium
    return model
model_cf_med = build_medium(model_cf)

[-0.005, -0.0035416666666666665, -0.0035416666666666665, -0.0035416666666666665, -0.0013750000000000001, -0.0011250000000000001, -0.00625, -0.004166666666666667, -0.5416666666666666, -0.041666666666666664, -0.05, -0.0013750000000000001]


In [69]:
opt(model_cf_med)

0.00118055555555
IN FLUXES      OUT FLUXES            OBJECTIVES
-------------  --------------------  ------------------
nh4_e  0.0417  na1_c       8.26      DM_btol_c  0.00118
               co2_c       0.335
               nh4_c       0.208
               h2o_c       0.167
               34hpp_c     0.00833
               phpyr_c     0.00833
               3mop_c      0.00833
               indole_p    0.00833
               3mob_c      0.00715
               cys__D_c    0.00417
               gdp_p       0.00354
               btol_c      0.00118
               3fe4s_c     0.000747
               12dgr160_c  0.000454
None


In [ ]:
alc_dehydr = cobra.Reaction(id='ALCDBUT', name='Alcohol dehydrogenase (butanal)', subsystem='c')
model.add_reaction(alc_dehydr)
alc_dehydr.add_metabolites({'btcoa_c': -1, 'h_c': -1, 'nadh_c': -1, 'nad_c': 1, 'btal_c': 1})

butanol = cobra.Metabolite(id='btol_c', name='1-butanol', compartment='c', charge=0, formula='C4H9OH')
but_synth = cobra.Reaction(id='BUTSYN', name='Butanol synthesis', subsystem='c')
model.add_reaction(but_synth)
but_synth.add_metabolites({'btal_c': -1, 'h_c': -1, 'nadh_c': -1, 'nad_c': 1, butanol: 1})

In [75]:
if (not 'DM_btol_c' in mt.reactions):
    mt.add_boundary(metabolite=mt.metabolites.btol_c, type='demand')
mt.objective = mt.reactions.DM_btol_c
mt.optimize()
mt.summary()

IN FLUXES    OUT FLUXES             OBJECTIVES
-----------  ---------------------  ------------------
             h_c         12.9       DM_btol_c  0.00118
             h2o_c        0.029
             gly_c        0.00951
             akg_c        0.00415
             gdp_p        0.00354
             dhor__S_c    0.00351
             xan_c        0.00181
             btol_c       0.00118
             12dgr120_c   0.00082
             fe3_c        0.000747
             3fe4s_c      0.000747
             gln__L_c     0.000632
             succ_c       0.00044
             cytd_c       0.00044
             iasp_c       0.000373
             3ophb_c      0.00011
             alaala_c     7.34e-05


In [80]:
mt.summary(fva=0.95)

IN FLUXES                  OUT FLUXES                                      OBJECTIVES
-------------------------  ----------------------------------------------  ------------------
id       Flux  Range       id                    Flux  Range               DM_btol_c  0.00118
-----  ------  ----------  ---------------  ---------  ------------------
k_e         0  [0, 0.542]  h_c              12.9       [0, 1e+03]
mg2_e       0  [0, 0.05]   h2o_c             0.029     [0, 1e+03]
                           gly_c             0.00951   [0, 0.213]
                           akg_c             0.00415   [0, 0.0864]
                           gdp_p             0.00354   [0, 0.00354]
                           dhor__S_c         0.00351   [0, 0.106]
                           xan_c             0.00181   [0, 0.0532]
                           btol_c            0.00118   [0.00112, 0.00118]
                           12dgr120_c        0.00082   [0, 0.00927]
                           fe3_c            

In [86]:
for r in mt.reactions:
    if r.flux > 0.01:
        print r.flux, r.compartments

0.0311436631945 set(['c'])
0.105903862847 set(['c'])
0.142601779514 set(['c'])
1000.0 set(['p', 'c'])
0.109268446181 set(['c'])
0.105903862847 set(['c'])
0.108013888889 set(['c'])
0.0166666666667 set(['p', 'c'])
3.15 set(['c'])
124.97021224 set(['p', 'c'])
1000.0 set(['p', 'e'])
60.0738841146 set(['c'])
0.0291666666667 set(['c'])
500.0 set(['p', 'c'])
1000.0 set(['p', 'e'])
1000.0 set(['p', 'e'])
1000.0 set(['p', 'c'])
1000.0 set(['p', 'e'])
1000.0 set(['p', 'c'])
1000.0 set(['p', 'e'])
0.281572916667 set(['c'])
0.281572916667 set(['c'])
0.312893663194 set(['c'])
1000.0 set(['p', 'e'])
1000.0 set(['p', 'c'])
0.0302881944445 set(['c'])
0.281572916667 set(['c'])
60.0761271701 set(['c'])
0.0125 set(['p', 'c'])
0.105903862847 set(['c'])
0.0105763888889 set(['c'])
0.105903862847 set(['c'])
1000.0 set(['p', 'e'])
999.979791667 set(['p', 'c'])
60.1643318142 set(['c'])
0.108013888889 set(['c'])
1000.0 set(['p', 'e'])
0.0311436631945 set(['c'])
0.0311436631945 set(['c'])
0.105903862847 set(['c'

In [30]:
df = pd.read_csv('./code/Karim_MetEng_2018_Figure2_Data.csv')
df.drop(columns=['Area_1', 'Area_2', 'Conc_1', 'Conc_2'], inplace=True)
df.head()

,AVG,STD,AVG.1,STD.1,Mg(Glu)2,NH4(Glu),K(Glu),Glucose,K2HPO4,NAD,ATP,CoA
0,0.00,0.0,0.00,0.00,8.0,10.0,134.0,0.0,10.0,0.5,0.0,0.5
1,0.00,0.0,0.00,0.00,8.0,10.0,134.0,200.0,10.0,0.0,0.0,0.5
2,0.00,0.0,0.00,0.00,8.0,10.0,134.0,200.0,10.0,0.5,0.0,2.0
3,0.00,0.0,0.00,0.00,8.0,10.0,134.0,200.0,10.0,0.5,0.0,1.6
4,1.21,1.7,0.09,0.13,8.0,10.0,134.0,200.0,10.0,0.5,2.0,0.5


In [60]:
addl_reagent_nms = ['mg2', 'nh4', 'k', 'glc__D', 'pi', 'nad', 'atp', 'coa']
for i, row in df.iterrows():
    if i != 4:
        continue
    print row
    with model_cf_med as mod:
        metab_dict = gen_metab_dict(mod, addl_reagent_nms, map(lambda x: -1 * conc_to_flux(x), row[4:]))
        #print metab_dict
        rxn = add_ex(mod, metab_dict, additive=True)
        if (not 'DM_btol_c' in mod.reactions):
            mod.add_boundary(metabolite=mod.metabolites.btol_c, type='demand')
        mod.objective = mod.reactions.DM_btol_c
        #mod.objective = {model_cf_med.reactions.Ec_biomass_iJO1366_core_53p95M: 1, model_cf_med.reactions.DM_btol_c: 1}
        #print mod.medium
        try:
            #sol = gapfill(mod, model, demand_reactions=False)
            print sol[0]
        except:
            print 'Fail'
        #mod.add_reactions(reaction_list=sol[0])
        obj = mod.optimize()
        print 'Obj: {0}'.format(obj.objective_value)
        #print mod.summary()
        #print mod.medium
        prod_env = production_envelope(mod, 'DM_btol_c')
        #prod_env = production_envelope(mod, map(lambda x: "EX_{0}_e".format(x), addl_reagent_nms))
        prod_env.head()
        if i >= 0:
            break
#mod.medium

AVG           1.21
STD           1.70
AVG.1         0.09
STD.1         0.13
Mg(Glu)2      8.00
NH4(Glu)     10.00
K(Glu)      134.00
Glucose     200.00
K2HPO4       10.00
NAD           0.50
ATP           2.00
CoA           0.50
Name: 4, dtype: float64
0.00021906891423811604
Obj: 0.0150694444444


In [171]:
model.exchanges

[<Reaction DM_4CRSOL at 0x10bae51d0>,
 <Reaction DM_5DRIB at 0x10bae53d0>,
 <Reaction DM_AACALD at 0x10bae5410>,
 <Reaction DM_AMOB at 0x10bae5450>,
 <Reaction DM_MTHTHF at 0x10bae5490>,
 <Reaction DM_OXAM at 0x10bae54d0>,
 <Reaction EX_12ppd__R_e at 0x10bae5590>,
 <Reaction EX_12ppd__S_e at 0x10bae55d0>,
 <Reaction EX_14glucan_e at 0x10bae5610>,
 <Reaction EX_15dap_e at 0x10bae5690>,
 <Reaction EX_23camp_e at 0x10bae5710>,
 <Reaction EX_23ccmp_e at 0x10bae5790>,
 <Reaction EX_23cgmp_e at 0x10bae5810>,
 <Reaction EX_23cump_e at 0x10bae5890>,
 <Reaction EX_23dappa_e at 0x10bae5910>,
 <Reaction EX_26dap__M_e at 0x10bae5950>,
 <Reaction EX_2ddglcn_e at 0x10bae5990>,
 <Reaction EX_34dhpac_e at 0x10bae59d0>,
 <Reaction EX_3amp_e at 0x10bae5a10>,
 <Reaction EX_3cmp_e at 0x10bae5a50>,
 <Reaction EX_3gmp_e at 0x10bae5a90>,
 <Reaction EX_3hcinnm_e at 0x10bae5ad0>,
 <Reaction EX_3hpp_e at 0x10bae5b10>,
 <Reaction EX_3hpppn_e at 0x10bae5b50>,
 <Reaction EX_3ump_e at 0x10bae5b90>,
 <Reaction EX_4a

In [189]:
addl_reagent_nms = ['mg2', 'nh4', 'k', 'glc', 'pi', 'nad', 'atp', 'coa']
with model_med as mod:
    for i, row in df.iterrows():
        metab_dict = gen_metab_dict(mod, addl_reagent_nms, map(lambda x: (x * 100) + 0.001, row[4:]))
        #print metab_dict
        rxn = add_ex(mod, metab_dict, additive=True)
        #print mod.medium
        try:
            sol = gapfill(mod, model, demand_reactions=False)
        except:
            print 'Fail'
            pass
        print sol[0]
        #mod.add_reactions(reaction_list=sol[0])
        obj = mod.optimize()
        print 'Obj: {0}'.format(obj.objective_value)
        mod.repair()
        if i >= 5:
            break

KeyError: 'glc_c'

In [29]:
model_med.add_reactions(reaction_list=[model.reactions.EX_pi_e, model.reactions.EX_pyr_e])

In [50]:
model_med.remove_reactions([model.reactions.EX_pi_e, model.reactions.EX_pyr_e])

In [ ]:
from cobra.flux_analysis import gapfill
sol = gapfill(model_med, model, demand_reactions=False)
model_med.add_reactions(reaction_list=sol[0])
sol[0]

In [30]:
if not ('DM_btol_c' in model_med.reactions):
    model_med.add_boundary(model_med.metabolites.btol_c, type='demand')
model_med.objective = model_med.reactions.DM_btol_c
obj = model_med.optimize()
obj.objective_value

1.0702040816325658

In [31]:
model_med.summary()

IN FLUXES         OUT FLUXES        OBJECTIVES
----------------  ----------------  ---------------
thr__L_e  105     gly_e     107     DM_btol_c  1.07
asp__L_e   91.4   pyr_e     101
asn__L_e   10.7   ala__L_e  100
ser__L_e    3.21  nh4_e       9.99
pi_e        3.21  glu__L_e    2.85
gln__L_e    2.14  btol_c      1.07
cys__L_e    1.07
val__L_e    1.07


In [36]:
df = pd.read_csv('./code/Karim_MetEng_2018_Figure2_Data.csv')
df.drop(columns=['Area_1', 'Area_2', 'Conc_1', 'Conc_2'], inplace=True)
df.head()

,AVG,STD,AVG.1,STD.1,Mg(Glu)2,NH4(Glu),K(Glu),Glucose,K2HPO4,NAD,ATP,CoA
0,0.00,0.0,0.00,0.00,8.0,10.0,134.0,0.0,10.0,0.5,0.0,0.5
1,0.00,0.0,0.00,0.00,8.0,10.0,134.0,200.0,10.0,0.0,0.0,0.5
2,0.00,0.0,0.00,0.00,8.0,10.0,134.0,200.0,10.0,0.5,0.0,2.0
3,0.00,0.0,0.00,0.00,8.0,10.0,134.0,200.0,10.0,0.5,0.0,1.6
4,1.21,1.7,0.09,0.13,8.0,10.0,134.0,200.0,10.0,0.5,2.0,0.5


In [149]:
df.values[0][4:].tolist()

[8.0, 10.0, 134.0, 0.0, 10.0, 0.5, 0.0, 0.5]

In [172]:
opt(model)

1.79035118836
IN FLUXES        OUT FLUXES     OBJECTIVES
---------------  -------------  ---------------
nh4_e     12.5   h2o_e   47.5   DM_btol_c  1.79
glc__D_e  10     co2_e   15.2
o2_e       9.87  h_e      5.37
pi_e       5.37  btol_c   1.79
so4_e      1.79
None


In [192]:
model_e.reactions.EX_glc_e

Reaction identifier,EX_glc_e
Name,D-Glucose exchange
Memory address,0x011b2c7990
Stoichiometry,glc__D_e <=> D-Glucose <=>
GPR,
Lower bound,-10.0
Upper bound,1000.0


In [191]:
model_e.medium

{'EX_ca2_e': 1000.0,
 'EX_cbl1_e': 0.01,
 'EX_cl_e': 1000.0,
 'EX_co2_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_glc_e': 10.0,
 'EX_h2o_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_na1_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_ni2_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_sel_e': 1000.0,
 'EX_slnt_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_tungs_e': 1000.0,
 'EX_zn2_e': 1000.0}

In [190]:
from cobra.flux_analysis.deletion import single_reaction_deletion
addl_reagent_nms = ['mg2', 'nh4', 'k', 'glc', 'pi', 'nad', 'atp', 'coa']
with model_e as mod:
    for i, row in df.iterrows():
        metab_dict = gen_metab_dict(mod, addl_reagent_nms, map(lambda x: (x * 100) + 0.001, row[4:]))
        #print metab_dict
        rxn = add_ex(mod, metab_dict, additive=False)
        print mod.medium
        try:
            pass
            #sol = gapfill(mod, model, demand_reactions=False)
        except:
            print 'Fail'
            pass
        #print sol[0]
        #mod.add_reactions(reaction_list=sol[0])
        obj = mod.optimize()
        print 'Obj: {0}'.format(obj.objective_value)
        if i >= 5:
            break

KeyError: 'glc_c'

In [184]:
delete_df[delete_df['growth'] < 1.790350]

,growth,status
ids,,
(BUTSYN),0.000000e+00,optimal
(ADSK),-2.325639e-14,optimal
(NH4tpp),-6.296851e-16,optimal
(GLUDy),1.721088e+00,optimal
(PGM),1.705204e+00,optimal
(PSERT),1.699087e+00,optimal
(GLCptspp),1.777739e+00,optimal
(TKT2),1.773866e+00,optimal
(PPKr),1.727273e+00,optimal


In [121]:
biomass = model_e.reactions.get_by_id('Ec_biomass_iJO1366_core_53p95M')

In [122]:
biomass.reactants

[<Metabolite arg__L_c at 0x10fd05850>,
 <Metabolite pe161_p at 0x10ffba890>,
 <Metabolite fe2_c at 0x10fd4f3d0>,
 <Metabolite nad_c at 0x10fdb00d0>,
 <Metabolite fad_c at 0x10fd4f0d0>,
 <Metabolite zn2_c at 0x10fe2a2d0>,
 <Metabolite val__L_c at 0x10fe1dc10>,
 <Metabolite pe161_c at 0x10fdbbf90>,
 <Metabolite cobalt2_c at 0x10fd1d950>,
 <Metabolite thr__L_c at 0x10fe05190>,
 <Metabolite h2o_c at 0x10fd72990>,
 <Metabolite nadp_c at 0x10fdb01d0>,
 <Metabolite nh4_c at 0x10fdb0350>,
 <Metabolite fe3_c at 0x10fd4f450>,
 <Metabolite mg2_c at 0x10fda3210>,
 <Metabolite his__L_c at 0x10fd7f250>,
 <Metabolite k_c at 0x10fd8a690>,
 <Metabolite cl_c at 0x10fd1d750>,
 <Metabolite so4_c at 0x10fdeca90>,
 <Metabolite pro__L_c at 0x10fdd3a90>,
 <Metabolite lys__L_c at 0x10fd982d0>,
 <Metabolite pheme_c at 0x10fdc89d0>,
 <Metabolite dttp_c at 0x10fd41810>,
 <Metabolite thf_c at 0x10fdfae50>,
 <Metabolite 2ohph_c at 0x10ed70350>,
 <Metabolite ca2_c at 0x10fd12890>,
 <Metabolite amet_c at 0x10fcf8990>

In [123]:
biomass.products

[<Metabolite adp_c at 0x10fb38cd0>,
 <Metabolite ppi_c at 0x10fdd3410>,
 <Metabolite h_c at 0x10fd72810>,
 <Metabolite pi_c at 0x10fdc8d50>]

In [61]:
biomass

Reaction identifier,Biomass_Ecoli_core
Name,Biomass Objective Function with GAM
Memory address,0x010edd5050
Stoichiometry,1.496 3pg_c + 3.7478 accoa_c + 59.81 atp_c + 0.361 e4p_c + 0.0709 f6p_c + 0.129 g3p_c + 0.205 g6p_c + 0.2557 gln__L_c + 4.9414 glu__L_c + 59.81 h2o_c + 3.547 nad_c + 13.0279 nadph_c + 1.7867 oaa_c ... 1.496 3-Phospho-D-glycerate + 3.7478 Acetyl-CoA + 59.81 ATP + 0.361 D-Erythrose 4-phosphate + 0.0709 D-Fructose 6-phosphate + 0.129 Glyceraldehyde 3-phosphate + 0.205 D-Glucose 6-phosphate + 0.2557...
GPR,
Lower bound,0.0
Upper bound,1000.0


In [16]:
import escher
import escher.urls

In [31]:
escher.list_available_models()

URLError: <urlopen error Could not contact Escher server>

In [30]:
print escher.list_available_maps()
d = escher.urls.root_directory
print('Escher directory: %s' % d)

URLError: <urlopen error Could not contact Escher server>

In [22]:
json_repr = cobra.io.to_json(model_e)

In [32]:
b = escher.Builder(model_json=json_repr, reaction_data=sol.fluxes.to_dict())
#b = escher.Builder(map_name='e_coli_core.Core metabolism', reaction_data=sol.fluxes.to_dict())
b.display_in_notebook()

In [ ]:
b = escher.Builder(map_name='e_coli_core.Core metabolism',
                   reaction_data=solution.x_dict,
                   # change the default colors
                   reaction_scale=[{'type': 'min', 'color': '#cccccc', 'size': 4},
                                   {'type': 'mean', 'color': '#0000dd', 'size': 20},
                                   {'type': 'max', 'color': '#ff0000', 'size': 40}],
                   # only show the primary metabolites
                   hide_secondary_metabolites=True)
b.display_in_notebook(js_source='local')

In [34]:
import Bio.KEGG as bk
from Bio.KEGG import REST
path = REST.kegg_find('PATHWAY', 'n-butanol')
path.readlines()

In [ ]:
re.match()

In [ ]:
rx = re.compile(r'^(?!.*(transport)).*$', re.IGNORECASE)

In [104]:
rx = re.compile(r'^(?!.*(transport|exchange|flippase|sink)).*$', re.IGNORECASE)
non_trans = eco.reactions.query(attribute='name', search_function=rx)
len(non_trans)

1556

In [106]:
i = 0
t = set()
for react in eco.reactions:
    if (react.name.find('transport') == -1 and react.subsystem.find('Transport') == -1 and react.name.find('exchange') == -1 and react.name.find('flippase') == -1  and react.name.find('Sink') == -1):
        i += 1
        t.add(react)
i

1556

In [112]:
from cobra import Model
import copy
eco = model_e.copy()
new_model = Model('new_model')

print("Adding compartment 'a' for 'all'...")
new_model.compartments['a'] = 'all'

#Adding all metabolites from existing model removing duplications
print("Adding all metabolites from existing model removing duplications...")
for x in eco.metabolites:
    dup = False
    for y in new_model.metabolites:
        if x.id[:-2] == y.id:
            dup = True
            break
    if dup == False:
        met = copy.deepcopy(x)
        met.id = met.id[:-2]+'_a'
        met.compartment = 'a'
        new_model.add_metabolites({met})

#Adding all reactions excluding transport, exchange, flippase, sink
print("Adding all reactions excluding transport, exchange, flippase, sink...")
rx = re.compile(r'^(?!.*(transport|exchange|flippase|sink)).*$', re.IGNORECASE)
non_trans = eco.reactions.query(attribute='name', search_function=rx)
for react in non_trans:
    for x in react.reactants:
        n = x.id[:-2]+'_a'
        if not new_model.metabolites.get_by_id(n):
            x.id = x.id[:-2]+'_a'
    for x in react.products:
        n = x.id[:-2]+'_a'
        if not new_model.metabolites.get_by_id(n):
            x.id = x.id[:-2]+'_a'
    new_model.add_reaction(react)
print("Number of reactions in new_model",len(new_model.reactions))

#removing all reactions that are dupliates
print("Removing all duplicate reactions...")
for react in new_model.reactions:
    for y in new_model.reactions:
        if react.reaction == y.reaction and react.name != y.name:
            y.delete()
print("Number of reactions after deleting duplicates: ",len(new_model.reactions))


# adding necessary metabolites and reactions for biosensor project
print("Adding necessary metabolites and reactions for biosensor project...")

Adding compartment 'a' for 'all'...
Adding all metabolites from existing model removing duplications...
Adding all reactions excluding transport, exchange, flippase, sink...
('Number of reactions in new_model', 1556)
Removing all duplicate reactions...
('Number of reactions after deleting duplicates: ', 1555)
Adding necessary metabolites and reactions for biosensor project...


In [114]:
alc_dehydr_a = cobra.Reaction(id='ALCDBUT', name='Alcohol dehydrogenase (butanal)', subsystem='a')
new_model.add_reaction(alc_dehydr_a)
alc_dehydr_a.add_metabolites({'btcoa_a': -1, 'h_a': -1, 'nadh_a': -1, 'nad_a': 1, 'btal_a': 1})

butanol_a = cobra.Metabolite(id='btol_a', name='1-butanol', compartment='a', charge=0, formula='C4H9OH')
but_synth_a = cobra.Reaction(id='BUTSYN', name='Butanol synthesis', subsystem='a')
new_model.add_reaction(but_synth_a)
but_synth_a.add_metabolites({'btal_a': -1, 'h_a': -1, 'nadh_a': -1, 'nad_a': 1, butanol: 1})

new_model.add_boundary(new_model.metabolites.btol_a, type='demand')

AttributeError: DictList has no attribute or entry btol_a

In [ ]:
new_model.metabolites.bto

In [113]:
new_model.objective = new_model.reaction.DM_btol_a
new_model.optimize()
new_model.summary()

Infeasible: None (infeasible)

In [30]:
m = cobra.io.load_json_model(filename='./code/varner.json')

In [58]:
m.metabolites.M_h_c_exchange_reverse

AttributeError: DictList has no attribute or entry M_h_c_exchange_reverse

In [59]:
m.reactions.M_h_c_exchange_reverse

Reaction identifier,M_h_c_exchange_reverse
Name,M_h_c_exchange_reverse
Memory address,0x01113c9c50
Stoichiometry,--> M_h_c --> M_h_c
GPR,
Lower bound,0
Upper bound,22.148976


In [34]:
m.objective = m.reactions.but_export

In [42]:
m.objective.expression

1.0*but_export - 1.0*but_export_reverse_3380a

In [67]:
help(m.problem.Model)

Help on class Model in module optlang.glpk_interface:

class Model(optlang.interface.Model)
 |  The model object represents an optimization problem and contains the variables, constraints an objective that
 |  make up the problem. Variables and constraints can be added and removed using the :code:`.add` and :code:`.remove` methods,
 |  while the objective can be changed by setting the objective attribute,
 |  e.g. :code:`model.objective = Objective(expr, direction="max")`.
 |  
 |  Once the problem has been formulated the optimization can be performed by calling the :code:`.optimize` method.
 |  This will return the status of the optimization, most commonly 'optimal', 'infeasible' or 'unbounded'.
 |  
 |  Attributes
 |  ----------
 |  objective: str
 |      The objective function.
 |  name: str, optional
 |      The name of the optimization problem.
 |  variables: Container, read-only
 |      Contains the variables of the optimization problem.
 |      The keys are the variable names an

In [66]:
m.problem.glp_write_lp(m.problem.Model, None, './code/py_prob.cplx')

TypeError: in method 'glp_write_lp', argument 1 of type 'glp_prob *'

In [69]:
import swiglpk
prob = swiglpk.glp_create_prob()

In [70]:
swiglpk.glp_read_lp(prob, None, './code/Sequence-Specific-FBA-CFPS-Publication-Code/Model/prob.cplx')

0

In [71]:
prob.

<Swig Object of type 'glp_prob *' at 0x110a7b720>

In [85]:
import optlang
from optlang import glpk_interface
optlang_prob = optlang.glpk_interface.glp_create_prob()

In [100]:
optlang.glpk_interface.glp_read_lp(optlang_prob, None, './code/Sequence-Specific-FBA-CFPS-Publication-Code/Model/prob.cplx')

0

In [101]:
opt_mod = glpk_interface.Model(problem=optlang_prob)

In [102]:
opt_mod.optimize()

'infeasible'

In [104]:
opt_mod.primal_values

OrderedDict([('R_220', -2.7859970478555176e-15),
             ('R_180', 0.0073605371900826826),
             ('R_179', 0.00736053719008264),
             ('R_177', 0.007360537190082639),
             ('R_176', 0.00736053719008264),
             ('R_183', 0.0011660478425183627),
             ('R_182', 0.00736053719008264),
             ('R_186', 0.0),
             ('R_185', 0.00736053719008264),
             ('R_174', 0.007360537190082638),
             ('R_173', 0.00736053719008264),
             ('R_147', 0.0),
             ('R_141', 0.0),
             ('R_129', 0.0),
             ('R_128', 0.0),
             ('R_13', 0.0),
             ('R_12', 9.713607064111928),
             ('R_11', 0.0),
             ('R_10', 9.713607064111928),
             ('R_38', 7.494748230761246),
             ('R_37', 7.494748230761244),
             ('R_17', 0.0),
             ('R_16', 9.71360706411193),
             ('R_15', 0.0),
             ('R_14', 9.713607064111928),
             ('R_169', -2.785997

In [16]:
import json

In [17]:
js = json.loads(cobra.io.to_json(model))

In [19]:
js.keys()

[u'reactions', u'genes', u'compartments', u'metabolites', u'version', u'id']

In [22]:
js['version']

u'1'

In [25]:
js['metabolites'][0]

{u'charge': -2,
 u'compartment': u'c',
 u'formula': u'C20H21N7O7',
 u'id': u'10fthf_c',
 u'name': u'10-Formyltetrahydrofolate'}

In [26]:
js['reactions']

[{u'gene_reaction_rule': u'',
  u'id': u'DM_4CRSOL',
  u'lower_bound': 0.0,
  u'metabolites': {u'4crsol_c': -1.0},
  u'name': u'Sink needed to allow p-Cresol to leave system',
  u'upper_bound': 1000.0},
 {u'gene_reaction_rule': u'',
  u'id': u'DM_5DRIB',
  u'lower_bound': 0.0,
  u'metabolites': {u'5drib_c': -1.0},
  u'name': u"Sink needed to allow 5'-deoxyribose to leave system",
  u'upper_bound': 1000.0},
 {u'gene_reaction_rule': u'',
  u'id': u'DM_AACALD',
  u'lower_bound': 0.0,
  u'metabolites': {u'aacald_c': -1.0},
  u'name': u'Sink needed to allow aminoacetaldehyde to leave system',
  u'upper_bound': 1000.0},
 {u'gene_reaction_rule': u'',
  u'id': u'DM_AMOB',
  u'lower_bound': 0.0,
  u'metabolites': {u'amob_c': -1.0},
  u'name': u'Sink needed to allow S-Adenosyl-4-methylthio-2-oxobutanoate to leave system',
  u'upper_bound': 1000.0},
 {u'gene_reaction_rule': u'',
  u'id': u'DM_MTHTHF',
  u'lower_bound': 0.0,
  u'metabolites': {u'mththf_c': -1.0},
  u'name': u'Sink needed to allow 

In [27]:
js['compartments']

{u'c': u'Cytoplasm', u'e': u'Extracellular', u'p': u'Periplasm'}

In [28]:
model.reactions.CDAPPA160

Reaction identifier,CDAPPA160
Name,CDP-Diacylglycerol pyrophostatase (n-C16:0)
Memory address,0x010f5a7590
Stoichiometry,"cdpdhdecg_c + h2o_c --> cmp_c + 2.0 h_c + pa160_c CDP-1,2-dihexadecanoylglycerol + H2O --> CMP + 2.0 H+ + 1,2-dihexadecanoyl-sn-glycerol 3-phosphate"
GPR,b3918
Lower bound,0.0
Upper bound,1000.0


In [105]:
mod_simp = cobra.test.create_test_model('textbook')

In [106]:
mod_simp

Name,e_coli_core
Memory address,0x010ee6dc90
Number of metabolites,72
Number of reactions,95
Objective expression,-1.0*Biomass_Ecoli_core_reverse_2cdba + 1.0*Biomass_Ecoli_core
Compartments,"cytosol, extracellular"
